In [ ]:
import os
import pypsa

import numpy as np
import networkx as nx
import pandas as pd
import random as rd
import cartopy.crs as ccrs
import matplotlib.pyplot as plt

from PIL import Image

import matplotlib
matplotlib.use('Agg')

In [ ]:
n = pypsa.examples.scigrid_de()
#n.add("Carrier", "Start")

#n.generators=n.generators.drop(index=n.generators.index)
    
colors = {'Solar': '#f9d002',
          'Wind Onshore': '#235ebc',
          'Wind Offshore': '#6895dd',
          'Battery': '#b8ea04',
          'Pumped Hydro': '#ea048a',
          'Storage Hydro': '#ea048a',
          'Hard Coal': '#707070',
          'Gas': '#23932d',
          'Run of River': '#4adbc8',
          'Waste': '#68896b',
          'Brown Coal': '#9e5a01',
          'Oil': '#262626',
          'Other': '#68896b',
          'Multiple': '#68896b',
          'Nuclear': '#ff9000',
          'Geothermal': '#ba91b1',
          'Start' : 'cadetblue'}

n.carriers.color = pd.Series(index=colors.keys(), data=colors.values())

buses_i = n.buses.query("carrier == 'AC'").index
n.madd("Generator", buses_i, " Start",
       bus=buses_i, carrier='Start', p_nom=1e-4);

In [ ]:
fig, ax = plt.subplots(figsize=(10,10))

#make plot(s) evenly sized
xmin = -.95; xmax = 1.05
ymin = -.95; ymax = .85
plt.plot([xmin, xmax, xmax, xmin, xmin], [ymin, ymin, ymax, ymax, ymin], 'o-', color='white', alpha=0)

n.plot(geomap=False, layouter=nx.drawing.layout.kamada_kawai_layout,
       bus_sizes=1e-4, line_widths=1, link_widths=1,
       transformer_widths=1,
       ax=ax)

plt.savefig(f"../pypsa.png", bbox_inches='tight', dpi=50)

plt.show()

In [ ]:
RE = ['Solar', 'Wind Onshore', 'Wind Offshore', 'Pumped Hydro', 'Storage Hydro']

bussizes_goal = n.generators.groupby(['bus', 'carrier']).sum().p_nom
bussizes_goal = bussizes_goal.append(n.storage_units.groupby(['bus', 'carrier']).sum().p_nom)/1e6
RE_idx = n.generators.query('carrier in @RE').groupby(['bus', 'carrier']).sum().index
RE_idx = RE_idx.append(n.storage_units.query('carrier in @RE').groupby(['bus', 'carrier']).sum().index)
bussizes_goal.loc[RE_idx]=0

bussizes_start = n.generators.groupby(['bus', 'carrier']).sum().p_nom
bussizes_start = bussizes_start.append(n.storage_units.groupby(['bus', 'carrier']).sum().p_nom)
S_idx = n.generators.query('carrier != "Start"').groupby(['bus', 'carrier']).sum().index
S_idx = S_idx.append(n.storage_units.query('carrier != "Start"').groupby(['bus', 'carrier']).sum().index)
bussizes_start.loc[S_idx]=0

linesizes_start = pd.Series(index=n.lines.index, data=1)
linesizes_goal = n.lines.s_nom/5e3

nframes_0 = 10

In [ ]:
(n.lines.s_nom/1e3).min()

In [ ]:
bussizes = bussizes_start
linesizes = linesizes_start

for i in range(nframes_0):
    fig,ax = plt.subplots(subplot_kw={"projection":ccrs.PlateCarree()}, figsize=(10,10))
    
    #make plot(s) evenly sized
    xmin = -.95; xmax = 1.05
    ymin = -.95; ymax = .85
    ax.plot([xmin, xmax, xmax, xmin, xmin], [ymin, ymin, ymax, ymax, ymin], 'o-', color='white', alpha=0)
    
    bussizes += (bussizes_goal-bussizes_start)/nframes_0
    linesizes += (linesizes_goal-linesizes_start)/nframes_0
        
    n.plot(ax=ax, geomap=False, layouter=nx.drawing.layout.kamada_kawai_layout,
           bus_sizes=bussizes, line_widths=linesizes)
    
    plt.savefig(f"../pypsa-{i}.png", bbox_inches='tight', dpi=50)
    print(f"pypsa-{i}.png ({i+1}/{nframes_0})")

#plt.show()

In [ ]:
nframes_1 = 10

bussizes_start = n.generators.groupby(['bus', 'carrier']).sum().p_nom
bussizes_start = bussizes_start.append(n.storage_units.groupby(['bus', 'carrier']).sum().p_nom)/1e6
RE_idx = n.generators.query('carrier in @RE').groupby(['bus', 'carrier']).sum().index
RE_idx = RE_idx.append(n.storage_units.query('carrier in @RE').groupby(['bus', 'carrier']).sum().index)
bussizes_start.loc[RE_idx]=0

bussizes_goal = n.generators.groupby(['bus', 'carrier']).sum().p_nom
bussizes_goal = bussizes_goal.append(n.storage_units.groupby(['bus', 'carrier']).sum().p_nom)/1e6
CO_idx = n.generators.query('carrier not in @RE').groupby(['bus', 'carrier']).sum().index
CO_idx = CO_idx.append(n.storage_units.query('carrier not in @RE').groupby(['bus', 'carrier']).sum().index)
bussizes_goal.loc[CO_idx]=0

linesizes_start = n.lines.s_nom/5e3
linesizes_goal = n.lines.s_nom/5e2

In [ ]:
#bussizes = bussizes_start
linesizes = linesizes_start

for i in range(nframes_1):
    fig,ax = plt.subplots(subplot_kw={"projection":ccrs.PlateCarree()}, figsize=(10,10))
    
    #make plot(s) evenly sized
    xmin = -.95; xmax = 1.05
    ymin = -.95; ymax = .85
    ax.plot([xmin, xmax, xmax, xmin, xmin], [ymin, ymin, ymax, ymax, ymin], 'o-', color='white', alpha=0)
    
    bussizes += (bussizes_goal-bussizes_start)/nframes_1
    linesizes += (linesizes_goal-linesizes_start)/nframes_1 + 0.25*(np.random.rand(1,len(linesizes))[0]-.5) #random noise from [-.25,.25)
    
    n.plot(ax=ax, geomap=False, layouter=nx.drawing.layout.kamada_kawai_layout,
           bus_sizes=bussizes, line_widths=linesizes)
    
    plt.savefig(f"../pypsa-{nframes_0+i}.png", bbox_inches='tight', dpi=50)
    print(f"pypsa-{nframes_0+i}.png ({i+1}/{nframes_1})")

#plt.show()

In [ ]:
#[i for i in range(nframes_0+nframes_1)]
#[i for i in range(10)]
#[i for i in range(nframes_0+nframes_1-2, -1, -1)]

In [ ]:
def gen_frame(path):
    im = Image.open(path)
    alpha = im.getchannel('A')

    # Convert the image into P mode but only use 255 colors in the palette out of 256
    im = im.convert('RGB').convert('P', palette=Image.ADAPTIVE, colors=255)

    # Set all pixel values below 128 to 255 , and the rest to 0
    mask = Image.eval(alpha, lambda a: 255 if a <=128 else 0)

    # Paste the color of index 255 and use alpha as a mask
    im.paste(255, mask)

    # The transparency index is 255
    im.info['transparency'] = 255
    return im


im1 = gen_frame('../pypsa.png')
append_images = np.append([f"../pypsa-{i}.png" for i in range(nframes_0+nframes_1)],
                          [f"../pypsa-{nframes_0+nframes_1-1}.png" for i in range(5)])
append_images = np.append(append_images, [f"../pypsa-{i}.png" for i in range(nframes_0+nframes_1-2, -1, -1)])
im1.save('../gifs/pypsa.gif', save_all=True, append_images=[gen_frame(append_images[i]) for i in range(len(append_images))],
         disposal=2, loop=0, duration=150)
im1.save('../pypsa.png', save_all=False, append_images=[])

In [ ]:
# delete images...
for file in [f"../pypsa-{i}.png" for i in range(0,nframes_0+nframes_1)]:
    os.remove(file) 